In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('../input/spotify-dataset-19212020-160k-tracks/data.csv')

In [ ]:
df.head()

(6)

In [ ]:
df.count()

On peut voir qu'il n'y a pas de valeurs manquantes dans les données. 

In [ ]:
dfnum = df.drop(['id','artists','key','explicit','mode','name','release_date','duration_ms'], axis = 1)


In [ ]:
dfnumgraph = dfnum.sample(frac = 0.001)

(7)

In [ ]:
sns.boxplot(x = dfnumgraph.energy ,data = dfnumgraph)

In [ ]:
sns.boxplot(x = dfnumgraph.instrumentalness ,data = dfnumgraph)

In [ ]:
sns.boxplot(x = dfnumgraph.danceability ,data = dfnumgraph)

In [ ]:
sns.boxplot(x = dfnumgraph.loudness ,data = dfnumgraph)

In [ ]:
sns.boxplot(x = dfnumgraph.speechiness ,data = dfnumgraph)

In [ ]:
plt.hist(dfnum.year, bins=99)

(8)

In [ ]:
%matplotlib inline
from matplotlib import pyplot
import plotly.express as px
fig = px.scatter_matrix(dfnumgraph)
fig.update_layout(
    title='Spotify',
    width=2000,
    height=2000)

fig.show()

(9)

In [ ]:
corr = dfnum.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

On cherche les variables les plus corrélés afin d'améliorer l'efficacité du clustering.

(10)

In [ ]:
sns.jointplot("acousticness", "energy", dfnumgraph, kind='kde')

In [ ]:
sns.jointplot("energy", "instrumentalness", dfnumgraph, kind='kde')

In [ ]:
sns.jointplot("loudness", "acousticness", dfnumgraph, kind='kde')

In [ ]:
sns.jointplot("danceability", "liveness", dfnumgraph, kind='kde')

(11)

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(dfnum)
kmeans_kwargs = {"init": "random",
       "n_init": 10,
      "max_iter": 300,
      "random_state": 42,}
   

sse = [] 
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
dfpetit = df.sample(frac = 0.05)
dfnumpetit = dfpetit.drop(['id','artists','key','explicit','mode','name','release_date','duration_ms'], axis = 1)


In [ ]:
L=list(dfpetit.name)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage 
import scipy.cluster.hierarchy as sch

(12)

In [ ]:
Z = linkage(dfnumpetit,method='ward',metric='euclidean') 
plt.figure(figsize=(12,12))
plt.title("CAH") 
dendrogram(Z,labels=dfnumpetit.index,no_labels=True,color_threshold=1200)
plt.show()

In [ ]:
groupes_cah = sch.fcluster(Z,t=1200,criterion='distance') 
print(np.unique(groupes_cah).size, "groupes constitués")

In [ ]:
dfnumpetit['name'] = L
dfnumpetit.head()

In [ ]:
idg = np.argsort(groupes_cah) 
gdf=dfnumpetit.iloc[idg]
gdf['groupe']=groupes_cah[idg]
gdf.groupby(['groupe'])
print(gdf)

In [ ]:
gdf.head()

In [ ]:
print(gdf.loc[gdf['groupe']==3,:])


In [ ]:
dfpetit.to_csv('dfpetit2.csv')

(13)

In [ ]:
print(gdf.loc[gdf['name']== 'Home Tonight',:])